In [ ]:
from geopy.geocoders import Nominatim
import pandas as pd
import re

In [ ]:
# Read data and add country series
original = pd.read_csv("new.csv")
original = original.drop(['StartDate', 'EndDate', 'Status', 'IPAddress', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'DistributionChannel', 'ExternalReference', 'UserLanguage'], axis=1)
first_two_rows = original.iloc[:2] 
original = original.iloc[2:]
original.rename(columns={'Text / Graphic': 'Consent', 'Q2 Single choice': 'Gender'}, inplace=True)

In [ ]:
# Drop columns that did not receive any replies
clean = original.dropna(axis=1, how='all')

In [ ]:
# Drop everyone who did not consent
condition_consent = original['Consent'] == 'Yes, I consent'
condition_nan = original.isnull().all(axis=1)
condition_consent = original[condition_consent | condition_nan]

In [ ]:
# Remove all unfinished participantions
condition_finished = original['Finished'] != 'False'
condition_nan = original.isnull().all(axis=1)

# Combine conditions using logical OR (|)
clean = original[condition_finished | condition_nan]

In [ ]:
# Add countries to dataset
geolocator = Nominatim(user_agent="example_app")

# Define a function for reverse geocoding
def reverse_geocode(row: pd.Series) -> str | None:
    """
    Reverse geocodes latitude and longitude values in a Pandas Series with 'geopy' to determine the country.

    Parameters:
        row (pd.Series): A Pandas Series containing latitude and longitude values.

    Returns:
        str | None: The country name if successfully reverse geocoded; otherwise, None.
    """
    if pd.isnull(row['LocationLatitude']) and pd.isnull(row['LocationLongitude']):
        return None
    location = geolocator.reverse((row['LocationLatitude'], row['LocationLongitude']), language='en')
    if location and 'address' in location.raw:
        return location.raw['address'].get('country', None)
    else:
        return None
    
# Apply the function to create a new 'Country' column
clean['Country'] = clean.apply(reverse_geocode, axis=1)


In [ ]:
# Change the times to appropriate european time
clean['RecordedDate'] = pd.to_datetime(clean['RecordedDate'])

# Define the time zone offset for America/Denver in hours
denver_offset = -7

# Convert to Central European Time (CET)
clean['RecordedDate'] = clean['RecordedDate'] + pd.DateOffset(hours=denver_offset)

In [ ]:
# Clean weekly listening data

def clean_series(series):
    cleaned_values = []

    for value in series:
        if isinstance(value, str):
            # Extract numeric values from strings
            numbers = re.findall(r'\d+', value)
            # If just normal numbers
            if numbers:
                # If multiple numbers are found, take the midpoint value for ranges
                num = int(numbers[0]) if len(numbers) == 1 else float(int(numbers[1]) + int(numbers[0]))/2.0

                # Check for additional characters and therefore special cases
                if 'h' in value:
                    cleaned_values.append(num)
                elif ',' in value:
                    cleaned_values.append(1.5)
                elif '.' in value:
                    cleaned_values.append(float(value))
                elif value == '5 hours a day x 7 days':
                    cleaned_values.append(5*7)
                elif 'daily' in value:
                    cleaned_values.append(num*7)
                else:
                    cleaned_values.append(num)
            else:
                cleaned_values.append(None)

        else:
            # If the value is not a string, keep it as is
            cleaned_values.append(value)

    return pd.Series(cleaned_values)

In [ ]:
# Weekly listening hours
clean.Q4 = clean_series(clean.Q4)
average_value = clean.Q4.mean()
print(f"On average participants listened {average_value} hours per week")

In [ ]:
clean.drop(['LocationLatitude','LocationLongitude','Consent', 'Finished'], axis=1, inplace=True)

In [ ]:
# Save cleaned data
clean.to_csv('clean.csv')